In [ ]:
folder_path = r'Documents'
text_loader_kwargs = {'autodetect_encoding':True}

In [31]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


folder_path = 'Documents'
text_loader_kwargs = {'autodetect_encoding':True}

# Might have to change the glob regex to look for different kinds of files
mixed_loader = DirectoryLoader(
    path = folder_path,
    loader_cls = TextLoader,
    loader_kwargs = text_loader_kwargs
)

doc = mixed_loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(doc)

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline
from langchain.llms.base import LLM

/Users/sushmey/Documents/GitHub/RAG_Venv3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
encoder =  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = FAISS.from_documents(documents=docs, embedding=encoder)
retriever = db.as_retriever(search_kwargs = {"k":10})

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12ade5b10>, search_kwargs={'k': 10})

In [23]:
from google.generativeai import configure, GenerativeModel
from dotenv import load_dotenv

load_dotenv()
configure(api_key=os.getenv("GEMINI_API_KEY"))

# Gemini wrapper as a LangChain-compatible LLM
class GeminiLLM(LLM):
    model: GenerativeModel

    def _call(self, prompt: str, stop=None):
        response = self.model.generate_content(prompt)
        return response.text

    @property
    def _llm_type(self):
        return "gemini"

In [24]:
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA
gemini_model = GenerativeModel("gemini-2.0-flash")
llm = GeminiLLM(model=gemini_model)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [29]:
query = "Is Nathan Ball a convicted rapist?"
print(qa.run(query))

Yes. The text states that Nathan Ball is appealing against his conviction on two counts of rape.

